In [1]:
import numpy as np
import pandas as pd

# 1.评估线性模型：

In [2]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

### 首先我们根据一些随机数据生成一个线性模型：

In [3]:
# 定义随机种子，使得结果可重复
rng = np.random.default_rng(seed=12345)

# 定义一个用于生成具有特定均值和方差的正态分布数据的辅助函数
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * rng.standard_normal(*size)

N = 100
# np.c_ 用于连接多个矩阵,X的shape为（100,3）
X = np.c_[dnorm(0, 0.4, size=N),
          dnorm(0, 0.6, size=N),
          dnorm(0, 0.2, size=N)]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps # 已知参数beta的 "真实 "模型

In [4]:
X[:5]

array([[-0.90050602, -0.18942958, -1.0278702 ],
       [ 0.79925205, -1.54598388, -0.32739708],
       [-0.55065483, -0.12025429,  0.32935899],
       [-0.16391555,  0.82403985,  0.20827485],
       [-0.04765129, -0.21314698, -0.04824364]])

### 线性模型一般是用截距项来拟合的，sm.add_constant 函数可以在现有的矩阵中添加截距项。

In [5]:
X_model = sm.add_constant(X)

In [6]:
X_model[:5]

array([[ 1.        , -0.90050602, -0.18942958, -1.0278702 ],
       [ 1.        ,  0.79925205, -1.54598388, -0.32739708],
       [ 1.        , -0.55065483, -0.12025429,  0.32935899],
       [ 1.        , -0.16391555,  0.82403985,  0.20827485],
       [ 1.        , -0.04765129, -0.21314698, -0.04824364]])

### m.OLS类可以拟合一个普通最小二乘法的线性回归。

In [7]:
model = sm.OLS(y, X)

### 模型的 fit 方法返回一个包含估计的模型参数和其他诊断的回归结果对象。

In [8]:
results = model.fit()

In [9]:
results.params

array([0.06681503, 0.26803235, 0.45052319])

### 在 results 上调用 summary 方法可以打印一个模型的详细诊断输出。

In [10]:
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.469
Model:                            OLS   Adj. R-squared (uncentered):              0.452
Method:                 Least Squares   F-statistic:                              28.51
Date:                Wed, 18 Jan 2023   Prob (F-statistic):                    2.66e-13
Time:                        18:52:51   Log-Likelihood:                         -25.611
No. Observations:                 100   AIC:                                      57.22
Df Residuals:                      97   BIC:                                      65.04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

### 上述输出中的参数名称已经被赋予了通用名称 x1，x2，等等。假设所有的模型参数都在一个DataFrame中。

In [11]:
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])

In [12]:
data['y'] = y

In [13]:
data[:5]

,col0,col1,col2,y
0,-0.900506,-0.189430,-1.027870,-0.599527
1,0.799252,-1.545984,-0.327397,-0.588454
2,-0.550655,-0.120254,0.329359,0.185634
3,-0.163916,0.824040,0.208275,-0.007477
4,-0.047651,-0.213147,-0.048244,-0.015374


### 现在我们可以使用statsmodels公式API和Patsy公式字符串。

In [14]:
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()

In [15]:
results.params

Intercept   -0.020799
col0         0.065813
col1         0.268970
col2         0.449419
dtype: float64

In [16]:
results.tvalues

Intercept   -0.652501
col0         1.219768
col1         6.312369
col2         6.567428
dtype: float64

### 给定新的样本外数据后，可以计算出给定的模型参数的预测值。

In [17]:
results.predict(data[:5])

0   -0.592959
1   -0.531160
2    0.058636
3    0.283658
4   -0.102947
dtype: float64

# 2.评估时间序列处理：

### 首先模拟生成一些具有自回归结构和噪声的时间序列数据：

In [18]:
init_x = 4

values = [init_x, init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

### 这个数据有一个AR（2）结构（两个滞后期），参数为0.8和-0.4。当你拟合一个AR模型时，你可能不知道要包括多少个滞后项，所以你可以用一些更大的滞后项来拟合模型。

In [19]:
from statsmodels.tsa.ar_model import AutoReg

In [20]:
MAXLAGS = 5

In [21]:
model = AutoReg(values, MAXLAGS)

In [22]:
results = model.fit()

### 结果中的估计参数首先是截距，然后是前两个滞后期的估计值。

In [23]:
results.params

array([ 0.02346612,  0.8096828 , -0.42865278, -0.03336517,  0.04267874,
       -0.05671529])